[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/seap-udea/multimin/blob/master/examples/multimin_quickstart.ipynb)

<p align="left"><img src="https://raw.githubusercontent.com/seap-udea/multimin/master/docs/multimin-logo-white.webp" width="600" /></p>

# MultiMin: Quickstart

This notebook provides a quick introduction to the package

## Installation and importing

If you're running this in Google Colab or need to install the package, uncomment and run the following cell:

In [ ]:
try:
    from google.colab import drive
    %pip install -Uq multimin
except ImportError:
    print("Not running in Colab, skipping installation")
    %load_ext autoreload
    %autoreload 2
!mkdir -p gallery/

# Uncomment to install from GitHub (development version)
# !pip install git+https://github.com/seap-udea/MultiMin.git

In [ ]:
import multimin as mn
import matplotlib.pyplot as plt
import plotly.graph_objects as go

import numpy as np
np.random.seed(1)
deg = np.pi/180

import warnings
warnings.filterwarnings("ignore")


## Roundtrip test

In this test, we will first generate a synthetic dataset using a CMND, and then we will fit this data.

Parameters of the Multivariate Gaussian Distribution:

In [ ]:
weights = [0.5,0.5]
mus = [[1.0, 0.5, -0.5], [1.0, -0.5, +0.5],]
sigmas = [[1, 1.2, 2.3], [0.8, 0.2, 3.3]]
angles = [
    [10*deg, 30*deg, 20*deg],
    [-20*deg, 0*deg, 30*deg],
] 
Sigmas = mn.Stats.calc_covariance_from_rotation(sigmas, angles)

Create the distribution object:

In [ ]:
CMND = mn.ComposedMultiVariateNormal(mus=mus, weights=weights, Sigmas=Sigmas)

Generate random samples from this distribution:

In [ ]:
data = CMND.rvs(5000)

Visualize the data:

In [ ]:
properties=dict(
    x=dict(label=r"$x$",range=None),
    y=dict(label=r"$y$",range=None),
    z=dict(label=r"$z$",range=None),
)

G=mn.CornerPlot(properties,figsize=3)
hargs=dict(bins=30,cmap='Spectral_r')

sargs=dict(s=1.2,edgecolor='None',color='r')
hist=G.scatter_plot(data,**sargs)

plt.savefig('gallery/multimin_quickstart_1.png')


We can also show the distribution of data points in sample in 3D:

In [ ]:
fig = go.Figure(
    data=go.Scatter3d(
        x=data[:, 0],
        y=data[:, 1],
        z=data[:, 2],
        mode="markers",
        marker={
            "size": 2,
            "color": data[:, 2],
            "colorscale": "Viridis",
            "showscale": False,
        },
    )
)

fig.update_layout(
    scene={
        "xaxis_title": properties["x"]["label"].replace("$", ""),
        "yaxis_title": properties["y"]["label"].replace("$", ""),
        "zaxis_title": properties["z"]["label"].replace("$", ""),
        "aspectmode": "cube",
    },
    showlegend=False,
)

fig.show()

To fit the data, we first define a fit handler:

In [ ]:
F = mn.FitCMND(ngauss=1, nvars=3)

Run the fitting procedure:

In [ ]:
F.fit_data(data, advance=True)

Check the results:

In [ ]:
G=F.plot_fit(
    props=["x","y","z"],
    hargs=dict(bins=30,cmap='YlGn'),
    sargs=dict(s=0.2,edgecolor='None',color='r'),
    figsize=3
)

plt.savefig('gallery/multimin_quickstart_2.png')


Not good. Let's try with two gaussians:

In [ ]:
F = mn.FitCMND(ngauss=2, nvars=3)
F.fit_data(data, advance=True)
G=F.plot_fit(
    props=["x","y","z"],
    hargs=dict(bins=30,cmap='YlGn'),
    sargs=dict(s=0.2,edgecolor='None',color='r'),
    figsize=3
)

plt.savefig('gallery/multimin_quickstart_3.png')


Check the initial and final distributions:

In [ ]:
CMND.Sigmas, F.cmnd.Sigmas, F.cmnd.weights

For comparison, let's see the centers of the CMNDs, the original and the fitted one:

In [ ]:
CMND.mus, F.cmnd.mus

Although the order is not the same the center almost coincide!

---

**MultiMin** - Multivariate Gaussian fitting

© 2026 Jorge I. Zuluaga